In [ ]:
import pandas as pd
from pathlib import Path
import re
import numpy as np
import json
from tqdm import tqdm
from multiprocessing import Pool, cpu_count
from sklearn.decomposition import PCA

In [ ]:
timesteps = 10
conversion_constant = (2.0*16.0)/8192
folder = "labeled_dataset_downsample_200_filter"

In [ ]:
def extract_class(filename):
    match = re.search(r'benchy_\d+_(.*?)\.parquet\.gzip', filename)
    if match:
        return match.group(1)
    return None

In [ ]:
def window_using_rolling(df,window=44,step=22,columns=["accel_x","accel_y","accel_z"]):
    for column in set(columns):
        df[column] = df[column]*conversion_constant
    rolling = df[columns].rolling(window=window,step=step)
    return np.array(list(rolling)[2:])

In [ ]:
def image_extraction(dataset,label):
    p_data = np.empty((0,*dataset.shape[1:]))

    for sample in dataset:
        # Perform 2D Discrete Fourier Transform (DFT)
        dft_image = np.fft.fft2(sample)

        # Compute the magnitude spectrum (for visualization)
        magnitude_spectrum = np.log10(np.abs(dft_image)+1)
        p_data = np.vstack((p_data,np.reshape(magnitude_spectrum,(1,*magnitude_spectrum.shape))
    ))

    return p_data

In [ ]:
def feature_extraction(dataset,label):
    p_data = np.empty((0,9))

    for sample in dataset:
        pca = PCA(n_components=1)
        pca.fit(sample)
        data = np.hstack([np.mean(sample,axis=0), np.std(sample,axis=0), pca.components_[0]])
        p_data = np.vstack((p_data,data))

    label_column = np.ones((p_data.shape[0], 1))*label
    p_data = np.hstack((p_data,label_column))
    return p_data

In [ ]:
def process_data(dataset,label):
    p_data = np.empty((0,9))

    for sample in dataset:
        pca = PCA(n_components=1)
        pca.fit(sample)
        data = np.hstack([np.mean(sample,axis=0), np.std(sample,axis=0), pca.components_[0]])
        p_data = np.vstack((p_data,data))
    
    label_column = np.ones((p_data.shape[0], 1))*label
    p_data = np.hstack((p_data,label_column))
    return p_data

In [ ]:
df_all = pd.DataFrame()
list_classes = set()
for file in Path(folder).glob("benchy_*"):
    list_classes.add(extract_class(file.name))
print(list_classes)

class_index = {'healthy':0, 'temp_220':1, 'temp_230':2, 'nozzle_03':3,'nozzle_02':4, 'loose_head':5}
print(class_index)
with open(Path(folder)/"class_index.json", 'w') as json_file:
    json.dump(class_index, json_file)

# Image dataset data

In [ ]:
x_data = np.empty((0, timesteps, 44, 4))
y_data = np.empty((0, 1))

def process_file(file):
    df = pd.read_parquet(file)
    data = window_using_rolling(df, columns=["accel_x", "accel_y", "accel_z", "accel_x"])
    unique = df["class"].unique()
    if len(unique) != 1:
        print(f"failure: {file} - {unique}")
        return None
    else:
        unique = unique[0]
    
    processed_data = image_extraction(data, class_index[unique])
    processed_data2 = processed_data[processed_data.shape[0] % timesteps:, :]
    reshaped_data = processed_data2.reshape((
        processed_data2.shape[0] // timesteps,
        timesteps,
        *processed_data2.shape[1:],
    ))
    y_data_temp = np.ones((reshaped_data.shape[0], 1)) * class_index[unique]
    return reshaped_data, y_data_temp

def collect_results(result):
    global x_data, y_data
    if result is not None:
        reshaped_data, y_data_temp = result
        x_data = np.vstack((x_data, reshaped_data))
        y_data = np.vstack((y_data, y_data_temp))

with Pool(cpu_count()) as pool:
    files = list(Path(folder).glob("benchy_*"))
    for result in tqdm(pool.imap_unordered(process_file, files), total=len(files)):
        collect_results(result)

In [ ]:
np.save(Path(folder)/"processed_labeled_image_data",x_data,fix_imports=False)
np.save(Path(folder)/"processed_labeled_image_result",y_data,fix_imports=False)

# Feature Dataset

Normalize data

In [ ]:
accel_x = np.array([])
accel_y = np.array([])
accel_z = np.array([])
counter = dict()

for file in tqdm(list(Path(folder).glob("benchy_*"))):
    file_class = extract_class(file.name)
    df = pd.read_parquet(file)
    accel_x = np.concatenate([accel_x,df["accel_x"]*conversion_constant])
    accel_y = np.concatenate([accel_y,df["accel_y"]*conversion_constant])
    accel_z = np.concatenate([accel_z,df["accel_z"]*conversion_constant])

mean_accel_x = np.mean(accel_x)
mean_accel_y = np.mean(accel_y)
mean_accel_z = np.mean(accel_z)

std_accel_x = np.std(accel_x)
std_accel_y = np.std(accel_y)
std_accel_z = np.std(accel_z)

In [ ]:
timesteps = 10
x_data = np.empty((0, timesteps, 9))
y_data = np.empty((0, 1))

def process_file(file):
    df = pd.read_parquet(file)
    
    # Normalize accelerometer data
    df["accel_x"] = (df["accel_x"] * conversion_constant - mean_accel_x) / std_accel_x
    df["accel_y"] = (df["accel_y"] * conversion_constant - mean_accel_y) / std_accel_y
    df["accel_z"] = (df["accel_z"] * conversion_constant - mean_accel_z) / std_accel_z

    # Apply rolling window function
    data = window_using_rolling(df)
    
    unique = df["class"].unique()
    if len(unique) != 1:
        print(f"failure: {file} - {unique}")
        return None, None
    unique = unique[0]

    # Feature extraction
    processed_data = feature_extraction(data, class_index[unique])

    # Reshape the data
    processed_data2 = processed_data[processed_data.shape[0] % timesteps:, :-1]
    reshaped_data = processed_data2.reshape((
        processed_data2.shape[0] // timesteps,
        timesteps,
        *processed_data2.shape[1:],
    ))

    return reshaped_data, np.ones((reshaped_data.shape[0], 1)) * class_index[unique]

def collect_results(result):
    global x_data, y_data
    if result is not None:
        reshaped_data, y_data_temp = result
        x_data = np.vstack((x_data, reshaped_data))
        y_data = np.vstack((y_data, y_data_temp))

# Create a pool of workers equal to the number of CPU cores
with Pool(cpu_count()) as pool:
    files = list(Path(folder).glob("benchy_*"))
    futures = [pool.apply_async(process_file, (file,)) for file in files]
    
    for future in tqdm(futures):
        result = future.get()
        collect_results(result)

In [ ]:
np.save(Path(folder)/"processed_labeled_filtered_normalized_data",x_data,fix_imports=False)
np.save(Path(folder)/"processed_labeled_filtered_normalized_result",y_data,fix_imports=False)